# Data preparation

## Database Connection

We used a free service to host our database. The Database is in PostgreSQL.

In [1]:
import json
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# DB Credentials

with open("../config.json") as config_file:
    config = json.load(config_file)

host = config["db_host"]
user = config["db_user"]
password = config["db_password"]
database = config["db_database"]
schema = config["db_schema"]

In [3]:
connection = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = connection.cursor()

def execute(query):
    cursor.execute(query)
    connection.commit()
    return cursor.fetchall()

def fetch(query):
    cursor.execute(query)
    return cursor.fetchall()

SELECT = "SELECT * FROM " + schema + "." # + table_name 
INSERT = "INSERT INTO " + schema + "." # + table_name + " VALUES " + values
UPDATE = "UPDATE " + schema + "." # + table_name + " SET " + column_name + " = " + value
DELETE = "DELETE FROM " + schema + "."  # + table_name + " WHERE " + column_name + " = " + value

# Test data, year 10
TEST_DATA = "year = 10"
TRAIN_DATA = "year < 10"

In [4]:
awards_players = fetch(SELECT + "awards_players") # awards and prizes received by players across 10 seasons,
coaches = fetch(SELECT + "coaches") # all coaches who've managed the teams during the time period,
players = fetch(SELECT + "players") # details of all players,
players_teams = fetch(SELECT + "players_teams") # performance of each player for each team they played,
series_post = fetch(SELECT + "series_post") # series' results,
teams = fetch(SELECT + "teams") # performance of the teams for each season,
teams_post = fetch(SELECT + "teams_post") # results of each team at the post-season.

In [5]:
#save the data in a dataframe
awards_players_df = pd.DataFrame(awards_players, columns=['playerID', 'award', 'year', 'lgID'])
coaches_df = pd.DataFrame(coaches, columns=['coachID', 'year', 'tmID', 'lgID', 'stint', 'won', 'lost', 'post_wins', 'post_losses'])
players_df = pd.DataFrame(players, columns=['bioID', 'pos', 'firstseason', 'lastseason', 'height', 'weight', 'college', 'collegeOther', 'birthDate', 'deathDate'])
players_teams_df = pd.DataFrame(players_teams, columns=['playerID', 'year', 'stint', 'tmID', 'lgID', 'GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade', 'dq', 'PostGP', 'PostGS', 'PostMinutes', 'PostPoints', 'PostoRebounds', 'PostdRebounds', 'PostRebounds', 'PostAssists', 'PostSteals', 'PostBlocks', 'PostTurnovers', 'PostPF', 'PostfgAttempted', 'PostfgMade', 'PostftAttempted', 'PostftMade', 'PostthreeAttempted', 'PostthreeMade', 'PostDQ'])
series_post_df = pd.DataFrame(series_post, columns=['year', 'round', 'series', 'tmIDWinner', 'lgIDWinner', 'tmIDLoser', 'lgIDLoser', 'W', 'L'])
teams_df = pd.DataFrame(teams, columns=['year', 'lgID', 'tmID', 'franchID', 'confID', 'divID', 'rank', 'playoff', 'seeded', 'firstRound', 'semis', 'finals', 'name', 'o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB', 'won', 'lost', 'GP', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'min', 'attend', 'arena'])
teams_post_df = pd.DataFrame(teams_post, columns=['year', 'tmID', 'lgID', 'W', 'L'])

#make a dictionary with all the dataframes
dfs = {'awards_players_df': awards_players_df, 'coaches_df': coaches_df, 'players_df': players_df, 'players_teams_df': players_teams_df, 'series_post_df': series_post_df, 'teams_df': teams_df, 'teams_post_df': teams_post_df}

So with that we end our understanding phase.
Our main takeaways are:
- There are dead players in the players table. We should take that into account when doing the analysis.
- There are players that have not played any season of the seasons given. We should take that into account when doing the analysis. There are 338 players that have not played any season.
- There are no Null entries (although there values that are simply an empty string)
- There are some columns with the DataType "object", most of them being strings.
- There are binary objects (like confID and playoff, in the 'teams' table, with the values "Y" or "N") that could be substituted by a binary, as well as ternary objects (like the firstRound, semis and finals in the 'teams' table, with the values "W", "L" or "") that could also be transformed.
- There are players with no position and no college assigned ("").
- There are players with no date of birth in the record (0000-00-00).
- There is the need to do null value uniformization, as there are some columns with empty strings, others with default 0 values and other values that represent null.
- The height and weight variables have default 0 values and should be treated as null values.
- The number of games played by each team differs (there may be teams that are no longer playing), so we can't compare the number of wins and losses directly. Win percentage should be used.
- In terms of win percentage, it seems like a competitive league, with more than half of the teams having a win percentage of 50% or more, taking advantage of the worst teams. There is also just one team below 40% of wins.
- There are teams that are no longer playing.
- There are a lot of highly correlated variables.

## Preparing the data for the model

In this notebook, we will prepare the data for the model. Having done the understanding in the [previous notebook](understanding.ipynb), we will now prepare the data for the model. From the understanding we came to the following conclusions:

So with that we end our understanding phase.
Our main takeaways are:
- There are dead players in the players table. We should take that into account when doing the analysis.
- There are players that have not played any season of the seasons given. We should take that into account when doing the analysis. There are 338 players that have not played any season.
- There are no Null entries (although there values that are simply an empty string)
- There are some columns with the DataType "object", most of them being strings.
- There are binary objects (like confID and playoff, in the 'teams' table, with the values "Y" or "N") that could be substituted by a binary, as well as ternary objects (like the firstRound, semis and finals in the 'teams' table, with the values "W", "L" or "") that could also be transformed.
- There are players with no position and no college assigned ("").
- There are players with no date of birth in the record (0000-00-00).
- There is the need to do null value uniformization, as there are some columns with empty strings, others with default 0 values and other values that represent null.
- The height and weight variables have default 0 values and should be treated as null values.
- The number of games played by each team differs (there may be teams that are no longer playing), so we can't compare the number of wins and losses directly. Win percentage should be used.
- In terms of win percentage, it seems like a competitive league, with more than half of the teams having a win percentage of 50% or more, taking advantage of the worst teams. There is also just one team below 40% of wins.
- There are teams that are no longer playing.
- There are a lot of highly correlated variables.

After considering our takeaways, we will now prepare the data for the model. We will do the following:
- Remove the players that have not played any season, and, if a player died, remove the seasons after the death.
- Transform the binary objects into binary values.
- Transform the ternary objects into binary values. (where the third value is a null value - after the null uniformization these are considered as binary objects too)
- Null uniformization: transform the empty strings and default 0 values into null values.
- Analysis null values: analyze the null values and decide what to do with them.
- Calculate win percentage for each team and add it to the teams table.

We begin by excluding columns that consistently have identical values since they do not contribute any valuable information to the model. However, we will retain the 'first season' and 'last season' of a player, as we intend to populate them with data.

In [6]:
#Drop columns whose values are always the same
for df in dfs:
    for col in dfs[df].columns:
        if len(dfs[df][col].unique()) == 1 and col not in ['firstseason', 'lastseason'] :
            print(df, col)
            dfs[df].drop(col, inplace=True, axis=1)

awards_players_df lgID
coaches_df lgID
players_teams_df lgID
series_post_df lgIDWinner
series_post_df lgIDLoser
teams_df lgID
teams_df divID
teams_df seeded
teams_df tmORB
teams_df tmDRB
teams_df tmTRB
teams_df opptmORB
teams_df opptmDRB
teams_df opptmTRB
teams_post_df lgID


### Null uniformization

We identified the following columns that have null values, but are not identified as such:
- players: height, weight, birthDate, position, college, deathDate
- teams: firstRound, semis, finals

In [7]:
#If date == 00-00-00, replace with null (birthDate and deathDate)

dfs["players_df"]["birthDate"] = dfs["players_df"]["birthDate"].replace('00-00-00', None)
dfs["players_df"]["birthDate"] = dfs["players_df"]["birthDate"].replace('0000-00-00', None)

# If value == 0, replace with null (height, weight)

dfs["players_df"]["height"] = dfs["players_df"]["height"].replace(0, None)
dfs["players_df"]["weight"] = dfs["players_df"]["weight"].replace(0, None)

# If value == "", replace with null (college, collegeOther, firstRound, semis, finals)

dfs["players_df"]["college"] = dfs["players_df"]["college"].replace('', None)
dfs["players_df"]["collegeOther"] = dfs["players_df"]["collegeOther"].replace('', None)
dfs["teams_df"]["firstRound"] = dfs["teams_df"]["firstRound"].replace('', None)
dfs["teams_df"]["semis"] = dfs["teams_df"]["semis"].replace('', None)
dfs["teams_df"]["finals"] = dfs["teams_df"]["finals"].replace('', None)

### Remove the players that have not played any season

In [8]:
#players that have not played in the last 10 years
players_not_played = fetch("SELECT p.bioid FROM wnba.players p WHERE p.bioid not in (select pt.playerid  from wnba.players_teams pt)")
#Convert list of tuples to list
players_not_played = [i[0] for i in players_not_played]
players_not_played

['abrahta01w',
 'adairje01w',
 'adamsda01w',
 'adamsmi01w',
 'adubari99w',
 'aglerbr99w',
 'alberma01w',
 'alexaer01w',
 'allenso99w',
 'angelyv01w',
 'appelja01w',
 'artiska01w',
 'ayimmi01w',
 'bakerla01w',
 'bassmi01w',
 'becenry01w',
 'beckan99wc',
 'bellje01w',
 'berggas01w',
 'berrysh01w',
 'berubca01w',
 'bibbyhe01w',
 'bishoab01w',
 'bjedoni01w',
 'bjorkan01w',
 'bladerh01w',
 'boguemu01w',
 'boldeba01w',
 'bookeka01w',
 'bosweca01w',
 'bouceje01w',
 'bouchke01w',
 'boyerli99w',
 'bradlki01w',
 'brancli01w',
 'branzal01w',
 'branzge01w',
 'braxtja01w',
 'brcanra01w',
 'brelaje01w',
 'brownci01w',
 'brownde01w',
 'brownla01w',
 'brownle01w',
 'brucegr99w',
 'bryanjo01w',
 'burgehe01w',
 'burgehe02w',
 'byrdla01w',
 'cabezli01w',
 'cainke01w',
 'camba01w',
 'cartede01w',
 'cartesy01w',
 'cebriel01w',
 'chacoke01w',
 'chancva99w',
 'charlda01w',
 'charlti01w',
 'chaseso01w',
 'chatmda99w',
 'cheekjo01w',
 'cheruam01w',
 'chestfe01w',
 'chriska02w',
 'clarkal01w',
 'clarkma01w',
 '

In [9]:
#Print the number of players that have not played in the last 10 years, and the lenght of the 3 dataframes that contain the playerID
print("Number of players that have not played: ", len(players_not_played))
print("-------------------------------------------")
print("Number of values in the players_team_df: ", len(dfs['players_teams_df']['playerID'].unique()))
print("Number of values in the awards_players_df: ", len(dfs['awards_players_df']['playerID'].unique()))
print("Number of values in the players_df: ", len(dfs['players_df']['bioID'].unique()))

#Remove the players that have not played in the last 10 years
for df in dfs:
    if(df == 'players_teams_df' or df == 'awards_players_df'):
        dfs[df] = dfs[df][~dfs[df]['playerID'].isin(players_not_played)]
    if(df == 'players_df'):
        dfs[df] = dfs[df][~dfs[df]['bioID'].isin(players_not_played)]

#Print the number of players that have not played in the last 10 years, and the lenght of the 3 dataframes that contain the playerID
print('\n')
print("Number of values in the players_team_df: ", len(dfs['players_teams_df']['playerID'].unique()))
print("Number of values in the awards_players_df: ", len(dfs['awards_players_df']['playerID'].unique()))
print("Number of values in the players_df: ", len(dfs['players_df']['bioID'].unique()))

Number of players that have not played:  338
-------------------------------------------
Number of values in the players_team_df:  555
Number of values in the awards_players_df:  58
Number of values in the players_df:  893


Number of values in the players_team_df:  555
Number of values in the awards_players_df:  51
Number of values in the players_df:  555


### Populate first and last seasons of a player in the wnba

As we mentioned before we will populate first and last season of a player in the wnba. We will do this by looking at the seasons table and finding the first and last season of a player. We will then populate the first and last season of a player in the players table.

In [10]:
# Group the players_teams_df by 'playerID' to find the first and last seasons.
first_seasons = players_teams_df.groupby('playerID')['year'].min()
last_seasons = players_teams_df.groupby('playerID')['year'].max()

# Update the 'firstseason' and 'lastseason' columns in players_df.
players_df['firstseason'] = players_df['bioID'].map(first_seasons)
players_df['lastseason'] = players_df['bioID'].map(last_seasons)

print(players_df.head())

        bioID  pos  firstseason  lastseason height weight            college  \
0  abrahta01w    C          NaN         NaN   74.0    190  George Washington   
1  abrossv01w    F          2.0         9.0   74.0    169        Connecticut   
2  adairje01w    C          NaN         NaN   76.0    197  George Washington   
3  adamsda01w  F-C          NaN         NaN   73.0    239          Texas A&M   
4  adamsjo01w    C          4.0         4.0   75.0    180         New Mexico   

             collegeOther   birthDate   deathDate  
0                    None  1975-09-27  0000-00-00  
1                    None  1980-07-09  0000-00-00  
2                    None  1986-12-19  0000-00-00  
3  Jefferson College (JC)  1989-02-19  0000-00-00  
4                    None  1981-05-24  0000-00-00  


### Transform the binary objects into binary values

In [11]:
#Get all the binary columns from all the dataframes
binary_columns = []
for df in dfs:
    binary_columns = binary_columns + [(df, list(dfs[df].columns[dfs[df].nunique() == 2]))]

#Print the binary columns uniques values
for i in binary_columns:
    if(len(i[1]) < 0):
        continue

    for j in i[1]:
        print("-------")
        print(i[0], j)
        print(dfs[i[0]][j].unique())

-------
players_df deathDate
['0000-00-00' '2011-05-27']
-------
series_post_df W
[2 3]
-------
teams_df confID
['EA' 'WE']
-------
teams_df playoff
['N' 'Y']
-------
teams_df firstRound
[None 'L' 'W']
-------
teams_df semis
[None 'W' 'L']
-------
teams_df finals
[None 'L' 'W']
-------
teams_df GP
[34 32]


Death date being a binary value is a coincidence, as only one player (that has played in the seasons we have) has died.
GP is the number of games played and should also not be converted to binary, as we will need this value to calculate the win percentage. (it is only binary because seasons have been of 32 games or 34 games). The W value in the series_post represents the number of wins a team winned in the playoffs. All the playoffs games are in the best of 3 or 5, so the winning team wins 2 or 3 games.
The other binary values are binary and should be converted to binary.

In [12]:
#Convert the binary columns to 0 and 1 (confID, playoff, firstRound, semis, finals)

binary_columns = ["confID", "playoff", "firstRound", "semis", "finals"]

dfs["teams_df"]

for col in binary_columns:
    dfs["teams_df"][col] = dfs["teams_df"][col].replace('EA', 0)
    dfs["teams_df"][col] = dfs["teams_df"][col].replace('WE', 1)
    dfs["teams_df"][col] = dfs["teams_df"][col].replace('L', 0)
    dfs["teams_df"][col] = dfs["teams_df"][col].replace('W', 1)
    dfs["teams_df"][col] = dfs["teams_df"][col].replace('N', 0)
    dfs["teams_df"][col] = dfs["teams_df"][col].replace('Y',1)
    #change the type of the column to int
    dfs["teams_df"][col] = dfs["teams_df"][col].astype("Int64")


### Calculate win percentage

For each team, we want to calculate the following:
- Win percentage
- Loss percentage
- Wins at home percentage
- Losses at home percentage
- Wins away percentage
- Losses away percentage
- Conference wins percentage
- Conference losses percentage

In [13]:
#Calculate win percentage, loss percentage, wins at home percentage, losses at home percentage, wins away percentage, losses away percentage, wins at conference percentage, losses at conference percentage

dfs["teams_df"]["win_percentage"] = dfs["teams_df"]["won"] / (dfs["teams_df"]["won"] + dfs["teams_df"]["lost"])
dfs["teams_df"]["loss_percentage"] = dfs["teams_df"]["lost"] / (dfs["teams_df"]["won"] + dfs["teams_df"]["lost"])
dfs["teams_df"]["home_win_percentage"] = dfs["teams_df"]["homeW"] / (dfs["teams_df"]["homeW"] + dfs["teams_df"]["homeL"])
dfs["teams_df"]["home_loss_percentage"] = dfs["teams_df"]["homeL"] / (dfs["teams_df"]["homeW"] + dfs["teams_df"]["homeL"])
dfs["teams_df"]["away_win_percentage"] = dfs["teams_df"]["awayW"] / (dfs["teams_df"]["awayW"] + dfs["teams_df"]["awayL"])
dfs["teams_df"]["away_loss_percentage"] = dfs["teams_df"]["awayL"] / (dfs["teams_df"]["awayW"] + dfs["teams_df"]["awayL"])
dfs["teams_df"]["conference_win_percentage"] = dfs["teams_df"]["confW"] / (dfs["teams_df"]["confW"] + dfs["teams_df"]["confL"])
dfs["teams_df"]["conference_loss_percentage"] = dfs["teams_df"]["confL"] / (dfs["teams_df"]["confW"] + dfs["teams_df"]["confL"])

#Drop the columns that are not needed anymore
dfs["teams_df"] = dfs["teams_df"].drop(columns=['won', 'lost', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL'])

dfs["teams_df"]

,year,tmID,franchID,confID,rank,playoff,firstRound,semis,finals,name,...,attend,arena,win_percentage,loss_percentage,home_win_percentage,home_loss_percentage,away_win_percentage,away_loss_percentage,conference_win_percentage,conference_loss_percentage
0,9,ATL,ATL,0,7,0,<NA>,<NA>,<NA>,Atlanta Dream,...,141379,Philips Arena,0.117647,0.882353,0.058824,0.941176,0.176471,0.823529,0.100000,0.900000
1,10,ATL,ATL,0,2,1,0,<NA>,<NA>,Atlanta Dream,...,120737,Philips Arena,0.529412,0.470588,0.705882,0.294118,0.352941,0.647059,0.454545,0.545455
2,1,CHA,CHA,0,8,0,<NA>,<NA>,<NA>,Charlotte Sting,...,90963,Charlotte Coliseum,0.250000,0.750000,0.312500,0.687500,0.187500,0.812500,0.238095,0.761905
3,2,CHA,CHA,0,4,1,1,1,0,Charlotte Sting,...,105525,Charlotte Coliseum,0.562500,0.437500,0.687500,0.312500,0.437500,0.562500,0.714286,0.285714
4,3,CHA,CHA,0,2,1,0,<NA>,<NA>,Charlotte Sting,...,106670,Charlotte Coliseum,0.562500,0.437500,0.687500,0.312500,0.437500,0.562500,0.571429,0.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,6,WAS,WAS,0,5,0,<NA>,<NA>,<NA>,Washington Mystics,...,171501,Verizon Center,0.470588,0.529412,0.588235,0.411765,0.352941,0.647059,0.450000,0.550000
138,7,WAS,WAS,0,4,1,0,<NA>,<NA>,Washington Mystics,...,133255,Verizon Center,0.529412,0.470588,0.764706,0.235294,0.294118,0.705882,0.600000,0.400000
139,8,WAS,WAS,0,5,0,<NA>,<NA>,<NA>,Washington Mystics,...,133255,Verizon Center,0.470588,0.529412,0.470588,0.529412,0.470588,0.529412,0.400000,0.600000
140,9,WAS,WAS,0,6,0,<NA>,<NA>,<NA>,Washington Mystics,...,154637,Verizon Center,0.294118,0.705882,0.352941,0.647059,0.235294,0.764706,0.300000,0.700000


In [14]:
dfs["teams_df"].columns

Index(['year', 'tmID', 'franchID', 'confID', 'rank', 'playoff', 'firstRound',
       'semis', 'finals', 'name', 'o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm',
       'o_3pa', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to',
       'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa',
       'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk',
       'd_pts', 'GP', 'min', 'attend', 'arena', 'win_percentage',
       'loss_percentage', 'home_win_percentage', 'home_loss_percentage',
       'away_win_percentage', 'away_loss_percentage',
       'conference_win_percentage', 'conference_loss_percentage'],
      dtype='object')

In [15]:
connection.close()